<br><br><br>
<span style="color:red;font-size:60px">GraphFrames Assignment</span>
<br><br>
In this assignment, you need to do the following:
<li>Read the file 201710-citibike-tripdata.csv</li>
<li>Construct a graph with stations as vertices and trips between stations as edges</li>
<li>Vertex Ids are station numbers and Vertex attributes are station names</li>
<li>Edge attributes are trip duration (durations are in seconds)</li>
<li>Then answer the questions below</li>

In [1]:
%%init_spark
launcher.packages= ["graphframes:graphframes:0.8.3-spark3.4-s_2.12"]

In [2]:
//GraphFrame imports
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._


//GraphX imports
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD




Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.7.189:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1699076829907)
SparkSession available as 'spark'


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.graphframes._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


<br><br><br>
<span style="color:green;font-size:xx-large">Step 1: Construct the graph</span>
<br><br>
<li>read the data file and drop the header line</li>
<li>create a vertex dataframe (the union of start stations and end stations)</li>
<li>create an edge dataframe (the trips - start station id, end station id, duration)</li>
<li>create a GraphFrame</li>

In [3]:
val text = sc.textFile("/Users/hardeepjohar/Documents/Courses/Fall2022/AnalyticsOnTheCloud/Assignments/201710-citibike-tripdata.csv")
val data = text.mapPartitionsWithIndex{ (idx,iter) => if (idx == 0) iter.drop(1) else iter }
    .map(l => l.split(","))
//val data = data_noh.map(l => l.split(","))
val start_station = data.map(f => (f(3).toInt,f(4))).distinct
val end_station = data.map(f => (f(7).toInt,f(8))).distinct
val vertices = start_station.union(end_station).distinct.toDF("id","location")
val edges = data.map(l => (l(3).toInt,l(7).toInt,l(0).toDouble)).toDF("src","dst","duration")

val g = GraphFrame(vertices,edges)

text: org.apache.spark.rdd.RDD[String] = /Users/hardeepjohar/Documents/Courses/Fall2022/AnalyticsOnTheCloud/Assignments/201710-citibike-tripdata.csv MapPartitionsRDD[1] at textFile at <console>:37
data: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[3] at map at <console>:39
start_station: org.apache.spark.rdd.RDD[(Int, String)] = MapPartitionsRDD[7] at distinct at <console>:41
end_station: org.apache.spark.rdd.RDD[(Int, String)] = MapPartitionsRDD[11] at distinct at <console>:42
vertices: org.apache.spark.sql.DataFrame = [id: int, location: string]
edges: org.apache.spark.sql.DataFrame = [src: int, dst: int ... 1 more field]
g: org.graphframes.GraphFrame = GraphFrame(v:[id: int, location: string], e:[src: int, dst: int ... 1 more field])


In [4]:
edges.show

+----+----+--------+
| src| dst|duration|
+----+----+--------+
| 479| 478|   457.0|
| 279| 307|  6462.0|
| 504| 350|   761.0|
|3236|3233|  1193.0|
|2006| 469|  2772.0|
|3443| 468|   260.0|
| 478|3224|   715.0|
|3305| 305|   808.0|
| 284| 355|  1143.0|
| 432| 295|   527.0|
|3430|3090|   516.0|
|2006| 469|  2764.0|
| 470| 458|   439.0|
|3414|3346|   292.0|
|3328|3314|   242.0|
|3414|3346|   284.0|
|3430|3109|   573.0|
| 161| 537|  1289.0|
|2006| 469|  2739.0|
| 440| 445|   949.0|
+----+----+--------+
only showing top 20 rows



<br><br><br>
<span style="color:green;font-size:xx-large">Step 2: Basic questions</span>
<br><br>

<li>How many citibike stations are there in the network?</li>
<li>How many trips were made in the month in question?</li>
<li>How many trips started and ended at the same station?</li>
<li>How many station to station connections are there (at least one edge exists between station i and station j and i is not equal to j)?</li>
<li>Your code should print:</li>
<pre>
Total number of stations: 785
Total number of trips.  : 1897592
Trips that started and ended at the same station: 33245
Number of station to station connections: 107524
</pre>

In [4]:
def make_undirected_graph(g: GraphFrame) = {
    val u_edge_df = g.find("(a)-[]->(b)")
        .select($"a.id".as("src"),$"b.id".as("dst"))
        .withColumn("swap",when(col("src")<col("dst"),col("dst")))
        .withColumn("dst",
                    when(col("swap").isNotNull,col("src"))
                    .otherwise(col("dst")))
        .withColumn("src",
                    when(col("swap").isNotNull,col("swap"))
                   .otherwise(col("src")))
        .drop(col("swap"))
        .distinct
    val u_vertices_df = g.vertices
    val u_g = GraphFrame(u_vertices_df,u_edge_df)    
    u_g
}

make_undirected_graph: (g: org.graphframes.GraphFrame)org.graphframes.GraphFrame


In [5]:
println("Total number of stations: "+g.vertices.count)
println("Total number of trips.  : "+g.edges.count)
g.edges.createOrReplaceTempView("edgeDB")
val r1 = spark.sql("select * from edgeDB where src==dst")
println("Trips that started and ended at the same station: "+r1.count)
val r2 = make_undirected_graph(GraphFrame(vertices,spark.sql("select * from edgeDB where src!=dst"))).edges.count
println("Number of station to station connections: "+r2)

Total number of stations: 785
Total number of trips.  : 1897592
Trips that started and ended at the same station: 33245
Number of station to station connections: 107524


r1: org.apache.spark.sql.DataFrame = [src: int, dst: int ... 1 more field]
r2: Long = 107524


In [10]:
val text = sc.textFile("/Users/hardeepjohar/Documents/Courses/Fall2022/AnalyticsOnTheCloud/Assignments/201710-citibike-tripdata.csv").mapPartitionsWithIndex{(idx,iter) => if (idx==0) iter.drop(1) else iter}

val start_station_name = text.map(x=>x.split(",")).map(x=>(x(4),x(3).toInt))
val end_station_name = text.map(x=>x.split(",")).map(x=>(x(8),x(7).toInt))

val start_df = start_station_name.toDF("name","id")
val end_df = end_station_name.toDF("name","id")
val all_station = start_df.union(end_df).distinct()

val edge_info = text.map(x=>x.split(",")).map(x=>(x(3).toInt,x(7).toInt,x(0).toInt))

val vertex_df = all_station
val edge_df = edge_info.toDF("src","dst","attr")

val g_2 = GraphFrame(vertex_df,edge_df)

val undirected_g2 = make_undirected_graph(g_2)
val s2s_connection = undirected_g2.find("(a)-[]->(b)").filter("a.id!=b.id").count()

text: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[98] at mapPartitionsWithIndex at <console>:40
start_station_name: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[100] at map at <console>:42
end_station_name: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[102] at map at <console>:43
start_df: org.apache.spark.sql.DataFrame = [name: string, id: int]
end_df: org.apache.spark.sql.DataFrame = [name: string, id: int]
all_station: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, id: int]
edge_info: org.apache.spark.rdd.RDD[(Int, Int, Int)] = MapPartitionsRDD[104] at map at <console>:49
vertex_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [name: string, id: int]
edge_df: org.apache.spark.sql.DataFrame = [src: int, dst: int ....


In [11]:
s2s_connection

res1: Long = 107524


<br><br><br>
<span style="color:green;font-size:xx-large">Step 3: Find the Station from which most trips originate</span>
<br><br>
<li>Note that the graph has one edge for each trip (i.e., there are many edges between pairs of vertices)</li>
<li>The function <span style="color:blue">outDegrees</span> returns the number of outgoing edges from every vertex</li>
<li>Print the name of the station with most originating trips</li>
<li>Your code should print:</li>
<pre>  
The station from which most trips originate is: "Pershing Square North"
</pre>

In [ ]:
val out = g.outDegrees
out.createOrReplaceTempView("outdegreesDB")
val maxout = spark.sql("select id from outdegreesDB where outDegree = (select max(outDegree) from outdegreesDB)")
println("The station from which most trips originate is: " + maxout.join(g.vertices,maxout("id") === g.vertices("id")).select("location").rdd.collect()(0)(0).toString)

<br><br><br>
<span style="color:green;font-size:xx-large">STEP 4: Proportion of trips for each station that start and end at that same station</span>
<br><br>
<li>Create a GraphX graph from the GraphFrames graph (use the method that retains datatypes)</li>
<li>Use aggregateMessages to calculate the number of trips that start and end at the same vertex (for each vertex)</li>
<li>Convert the resulting (VertexRDD) to a DataFrame</li>
<li>Using join add the location of the station column to the result df from the previous step and then use select to create a dataframe with the schema (vertex, location, trips)</li>
<li>Join this df to the out degrees df created earlier</li>
<li>Divide the "same trips" column by the "out degrees column" </li>
<li>Sort the resulting df by this proportion in descending order</li>
<li>Your output should be the following dataframe
<pre>
+----+--------------------+-----+---------+-------------------+
|  id|            location|trips|outDegree|               prop|
+----+--------------------+-----+---------+-------------------+
|3488|  "8D QC Station 01"|    1|        1|                1.0|
|3245|"NYCBS DEPOT - DE...|    1|        2|                0.5|
|3182|"Yankee Ferry Ter...|  309|      900| 0.3433333333333333|
|3254|  "Soissons Landing"|  358|     1100|0.32545454545454544|
|3342|"Pioneer St & Ric...|   59|      299|0.19732441471571907|
|3477|"39 St & 2 Ave - ...|   45|      245| 0.1836734693877551|
|3532|"Ditmars Blvd & 1...|   70|      407|  0.171990171990172|
|3180|"Brooklyn Bridge ...|  232|     1354|0.17134416543574593|
|3423|"West Drive & Pro...|  367|     2463|0.14900527811611855|
|3636|"Expansion Wareho...|    1|        8|              0.125|
|3302|"Columbus Ave & W...|   74|      598|0.12374581939799331|
|3120|"Center Blvd & Bo...|   74|      622| 0.1189710610932476|
|3514|"Astoria Park S &...|   34|      299|0.11371237458193979|
|3479|      "Picnic Point"|   77|      712|0.10814606741573034|
|3594|"Montgomery St & ...|    9|       87|0.10344827586206896|
|3524|    "19 St & 24 Ave"|   24|      249| 0.0963855421686747|
|3349|"Grand Army Plaza...|  237|     2570|0.09221789883268483|
|3333|"Columbia St & Lo...|    5|       55|0.09090909090909091|
|3354|"3 St & Prospect ...|  142|     1572|0.09033078880407125|
|3607|    "31 Ave & 14 St"|   10|      113|0.08849557522123894|
+----+--------------------+-----+---------+-------------------+
only showing top 20 rows
</pre>

In [ ]:
val v = g.vertices.rdd.map(r => (r(0).toString.toLong,r(1).toString))
val e = g.edges.rdd.map(r => Edge(r(0).toString.toLong,r(1).toString.toLong,r(2).toString.toDouble))
val gx: Graph[String, Double] = Graph(v, e)

In [ ]:
val same_trips = gx.aggregateMessages[Int](ec => ec.sendToSrc(if (ec.srcId == ec.dstId) 1 else 0),(x,y) => x+y)
                    .toDF("vertex","trips")
same_trips.join(g.vertices,g.vertices("id")===same_trips("vertex"))
            .select("vertex","location","trips")
            .join(out,same_trips("vertex")===out("id")).select("id","location","trips","outDegree")
            .withColumn("prop",col("trips")/col("outDegree"))
            .orderBy(desc("prop"))
            .show

<br><br><br>
<span style="color:green;font-size:xx-large">STEP 5: Create a new graph that contains all edges except for those between the same station</span>
<br><br>


In [ ]:
val new_g = g.filterEdges("src != dst")

In [ ]:
new_g.edges.show

<br><br><br>
<span style="color:green;font-size:xx-large">STEP 6: Calculate the average duration between every pair of stations</span>
<br><br>
<li>use the new graph from step 5 for this</li>
<li>I'll let you figure this out but this should be really easy (think SQL)</li>
<pre>
+----+----+------------------+
| src| dst|                 m|
+----+----+------------------+
| 504| 350| 772.7647058823529|
| 433| 527| 532.9677419354839|
| 434| 470|316.52272727272725|
| 438| 151|  546.195652173913|
| 445| 507| 553.3947368421053|
|2021| 446| 827.6904761904761|
| 116| 518| 1115.857142857143|
|3435| 358| 895.6666666666666|
|3402|3414| 634.1666666666666|
| 498| 495| 801.2272727272727|
|3637| 418|             889.7|
| 380|3260|419.42105263157896|
|3360| 507|            1442.0|
| 326| 247|             713.0|
|3358| 467| 500.6666666666667|
|3164| 457|502.97241379310344|
| 498| 528| 434.3207547169811|
| 405|3256| 843.2168674698795|
| 477|2000|2672.3333333333335|
|3226|3163| 686.4444444444445|
+----+----+------------------+
only showing top 20 rows
<pre>

In [ ]:
new_g.edges.createOrReplaceTempView("edgesDB")
val result = spark.sql("select src,dst,mean(duration) as m from edgesDB group by src,dst")
result.show

<br><br><br>
<span style="color:green;font-size:xx-large">STEP 7: Important stations</span><br><br>
Citibike wants to figure out how best to deploy its workers in checking whether a station is over-full (too many bikes) or needs more bikes. It figures that the best way to do this is to find out which stations are the most important in terms of flows:
<li>A station that has high bike returns and is connected to other stations with high bike returns is more likely to have too many bikes in its station and therefore should be monitored more often</li>
<li>A station that has high bike pickups and is connected to other stations with high bike pickups is more likely to be short of bikes and therefore should be monitored more often</li>
<li>Calculate the propensities for over-fullness and emptiness for every station</li>
<li>Report the 5 most important stations for over-fullness (use pageRank on the graph)</li>
<li>Report the 5 most important stations for emptiness (reverse all the edges on the graph and use pageRank)</li>
<li>Your results (Don't worry about the meaning of location names!):</li>
<pre>
+---+--------------------+------------------+
| id|            location|          pagerank|
+---+--------------------+------------------+
|519|"Pershing Square ...| 4.930887390071603|
|426|"West St & Chambe...|3.7410934274030576|
|402|"Broadway & E 22 St"|  3.58520147183096|
|497|"E 17 St & Broadway"| 3.537658018512581|
|435|   "W 21 St & 6 Ave"| 3.438585855241344|
+---+--------------------+------------------+

+----+--------------------+------------------+
|  id|            location|          pagerank|
+----+--------------------+------------------+
|3197|      "Hs Don't Use"| 5.710640869520747|
| 519|"Pershing Square ...| 5.012823444592195|
|3480|      "WS Don't Use"| 4.272284643284593|
| 402|"Broadway & E 22 St"|3.4515211069038183|
| 497|"E 17 St & Broadway"|3.3347259745457443|
+----+--------------------+------------------+
</pre>


In [ ]:
val ranks = g.pageRank.resetProbability(0.15).tol(0.0001)
                .run()
                .vertices
                .orderBy(desc("pagerank"))
                .limit(5)
                .show


In [ ]:
val e2 = g.edges.withColumn("temp",col("src"))
        .withColumn("src",col("dst"))
        .withColumn("dst",col("temp"))
        .select("src","dst","duration")
val g2 = GraphFrame(vertices,e2)
val ranks = g2.pageRank.resetProbability(0.15).tol(0.0001)
                .run()
                .vertices
                .orderBy(desc("pagerank"))
                .limit(5)
                .show



<br><br><br>
<span style="color:green;font-size:xx-large">STEP 8: Calculate the clustering coefficient of every station</span><br><br>

<li>Find the number of triangles that each vertex belongs to in the undirected graph</li>
<li>Get the number of adjacent vertices (degrees of the undirected graph)</li>
<li>Calculate the number of possible triangles a vertex can belong to (for every vertex)</li>
<li>Divide actual triangles by possible triangles for each vertex

<li>And report the top 20 stations by clustering coefficient</li>
<pre>
+----+--------------------+------------------+
|  id|            location|             coeff|
+----+--------------------+------------------+
|3040|     "GOW Tech Shop"|               1.0|
|3639|        "Harborside"|               1.0|
|3192|"Liberty Light Rail"|               1.0|
|3485| "NYCBS Depot - RIS"|               1.0|
|3647|    "48 Ave & 30 Pl"|               1.0|
|3279|       "Dixon Mills"|               1.0|
|3186|     "Grove St PATH"|               1.0|
| 153|   "E 40 St & 5 Ave"|               1.0|
| 339|"Avenue D & E 12 St"| 0.877201420748853|
|3464|"W 37 St & Broadway"|0.8679573382796197|
| 247|"Perry St & Bleec...|0.8602079768329604|
|3175|"W 70 St & Amster...|0.8592469808193227|
|3176|"W 64 St & West E...|0.8568452539928423|
|3623|"W 120 St & Clare...|0.8549019607843137|
|3491|  "E 118 St & 1 Ave"| 0.854122621564482|
| 266| "Avenue D & E 8 St"| 0.849218980253463|
|3441|   "10 Hudson Yards"|0.8482701509017299|
|3646|    "35 Ave & 10 St"|0.8333333333333334|
|3642|"E 98 St & Lexing...|             0.832|
| 444|"Broadway & W 24 St"|0.8283229697508064|
+----+--------------------+------------------+
only showing top 20 rows
</pre>

In [ ]:
val triangles = g.triangleCount.run().withColumnRenamed("id","t_id") //Get the number of triangles each vertex belongs to
val degrees = make_undirected_graph(g).degrees //Get the number of adjacent vertices for each vertex
val possible = degrees.withColumn("possible",col("degree")*(col("degree")-1)/lit(2)) //Calculate possible triangles
val joined = triangles.select("t_id","count").join(possible,triangles("t_id")===possible("id")) 
val coeff = joined.withColumn("coeff",col("count")/col("possible")).select("t_id","coeff")
coeff.join(g.vertices,coeff("t_id")===g.vertices("id"))
    .orderBy(desc("coeff"))
    .select("id","location","coeff")
    .show
